In [ ]:
# Interactive Figure
# %matplotlib ipympl 
# Non interactive
%matplotlib inline 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def addjob(qmprog, qm):
    # Add a QUA program to the OPX queue, which compiles it and executes it
    job = qm.queue.add(qmprog)
    # Wait for job to be loaded
    while job.status=="loading":
        print("Job is loading...")
        time.sleep(0.1)
    # Wait until job is running
    time.sleep(0.1)
    while job.status=="pending":
        q = job.position_in_queue()
        if q>0:
            print("Position in queue",q,end='\r')
        time.sleep(0.1)
    job=job.wait_for_execution()
    print("\nJob is running")
    return job

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets

class JobAborter:
    def __init__(self,job,timelimit=10):
        self.job=job
        self.keeprunning=True
        self.progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
        self.button = widgets.Button(description='Abort',)
        self.start_time = time.time()
        self.button.on_click(self.stop)
        self.timelimit = timelimit
        self.thread = threading.Thread(target=self.update,)
        self.thread.start()
    def stop(self,b):
        self.keeprunning = False
    def update(self):
        dt = 0
        while dt<1 and self.keeprunning and job.status=='running':
            dt = (time.time()-self.start_time)/self.timelimit
            self.progress.value = dt
            time.sleep(0.1)
        self.job.halt()
        print('Job finished')
    def show(self):
        display(widgets.HBox([self.button,self.progress,]))

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Get running QM instance

In [ ]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Send Pulses and Visualize them

In [ ]:
with program() as prog:
    with infinite_loop_():
        play('trigger','trigger')
        # Pulse sequence
        play('pulse','qubit',duration=400*u.ns)
        wait(200*u.ns)
        play('pulse','qubit',duration=400*u.ns)
        wait(10*u.us)

# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()

# Practice

## Exercise 1: Visualize a gaussian pulse
- Replace one square pulse by a gaussian pulse.

In [ ]:
with program() as prog:
    with infinite_loop_():
        play('trigger','trigger')
        # Pulse sequence
        play('pulse','qubit',duration=400*u.ns)
        wait(200*u.ns)
        play('pulse','qubit',duration=400*u.ns)
        wait(10*u.us)

# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()

## Exercise 2: Vary delay in a loop 
- Change the delay between two square pulses in a loop. Note the useful command to loop over a Numpy array.
- Check that you get what you want on the spectrum analyzer. 

In [ ]:
durations = np.arange(200)
with program() as prog:
    t=declare(int)
    with infinite_loop_():
        with for_(*from_array(t, durations)):
            play('trigger','trigger')
            # Pulse sequence
            play('pulse','qubit',duration=400*u.ns)
            wait(t)
            play('pulse','qubit',duration=400*u.ns)
            wait(10*u.ms)

# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()

## Exercise 3: Vary pulse duration
- Change the duration of the second pulse in a loop. 
- Check that you get what you want on the spectrum analyzer. 

In [ ]:
durations = np.arange(100,200)
with program() as prog:
    t=declare(int)
    with infinite_loop_():
        with for_(*from_array(t, durations)):
            play('trigger','trigger')
            # Pulse sequence
            play('pulse','qubit',duration=200*u.ns)
            wait(400*u.ns)
            play('pulse','qubit',duration=200*u.ns)
            wait(10*u.ms)

# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()

## Exercise 4: Vary pulse amplitude in a loop 
- Change the amplitude of the second pulse in a loop.
- Check that you get what you want on the spectrum analyzer. 

In [ ]:
amplitudes = np.arange(0,1,0.001)
with program() as prog:
    a=declare(fixed)
    with infinite_loop_():
        with for_(*from_array(a, amplitudes)):
            play('trigger','trigger')
            # Pulse sequence
            play('gaussian','qubit')
            wait(400*u.ns)
            play('gaussian','qubit')
            wait(10*u.ms)

# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()